In [56]:
# import relevant libraries
from bs4 import BeautifulSoup
import requests
import json
import os
import time

import pandas as pd
import regex as re
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

# get latest Chrome driver and maximize screen
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

C:\Users\bartb\AppData\Local\Temp\ipykernel_28360\555813881.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [57]:
# Get the chromedriver.
PATH = 'C:/Users/bartb/Downloads/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(PATH)

C:\Users\bartb\AppData\Local\Temp\ipykernel_28360\3124890759.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [65]:
# Go to the Tripadvisor website for London England.
# And because we're scraping english restaurants, go to co.uk site
url = 'https://www.tripadvisor.co.uk/Restaurants-g186337-Liverpool_Merseyside_England.html'
driver.get(url)
time.sleep(3)
# Click OK for cookies if present
try:
    driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
except:
    pass
time.sleep(2)
# Divide the amount of restaurants by 30, round upwards. This is the amount of restaurants it will be scraping.
pages = int(int(driver.find_element(By.XPATH,"*//span[@class = 'b']").text)/30)
pages

49

In [68]:
%%time
i = 1
links = []
while i <= pages:
    html = driver.page_source # get the page source
    soup = BeautifulSoup(html,'html.parser') # transform to BeautifulSoup object
    links.append(soup.find_all(class_='Lwqic Cj b')[1]['href']) #get the first non-ad restaurant link (top/bottom are ads)
    time.sleep(1)
        
    # Scroll to the element, otherwise it might fail
    target = driver.find_element(By.XPATH,"*//a[@class = 'nav next rndBtn ui_button primary taLnk']")
    actions = ActionChains(driver)
    actions.move_to_element(target)
    actions.perform()
    
    # If its the last page, the button won't be there, thus fail.
    try:    
        target.click()
        time.sleep(3)
    except:
        break
        
    i=i+1
links

['/Restaurant_Review-g186337-d23837575-Reviews-Blackdoor_Bar_Bistro-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d17456232-Reviews-Bundobust_Liverpool-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d23332799-Reviews-Mamasan_Liverpool-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d2399951-Reviews-Greek_Taverna_Maghull-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d7599071-Reviews-BrewDog_Liverpool-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d3610248-Reviews-Spice_City-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d13753701-Reviews-Mays_Thai_Kitchen-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d1552716-Reviews-Stag_and_Rainbow-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d15873030-Reviews-Hope_and_Smoke-Liverpool_Merseyside_England.html',
 '/Restaurant_Review-g186337-d21084989-Reviews-The_Vale_Bar_Mo_s_Bistro-Liverpool_Merseyside_Engl

In [81]:
%%time
review_all = pd.DataFrame()
# For all extracted links get their reviews.
for i in links: #Set less if desired
    driver.get('https://www.tripadvisor.co.uk'+i) # Add the links to the tripadvisor url.
    time.sleep(2)
    driver.find_element(By.ID,"filters_detail_language_filterLang_en").click() # We only use English language reviews
    time.sleep(2)
    
    # Scroll to the element, otherwise it might fail
    target = driver.find_element(By.XPATH,"*//*[@class ='taLnk ulBlueLinks']")
    actions = ActionChains(driver)
    actions.move_to_element(target)
    actions.perform()
    
    try: # Expand review text to get the whole text. If the review is short enough, the button won't be there.
        if target.text == 'More': 
            target.click()
    except:
        pass
               
    time.sleep(1)
    review_master = get_reviews(driver) # Scrape the reviews, self-built function see below.
    review_all = pd.concat([review_all, review_master])
review_all = review_all.reset_index(drop = True)

MaxRetryError: HTTPConnectionPool(host='localhost', port=50301): Max retries exceeded with url: /session/49c44969b3c276bbd8c287627efdc679/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B7C1706C40>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [75]:
# Rough categorization of cuisines plus if its a bar/pub or not
df = review_all
df['Italian'] = df['category'].str.contains('Italian|Pizza|Pasta|Lasagna')
df['East Asian'] = df['category'].str.contains('Chinese|Japanese|Sushi|Korean|Taiwanese')
df['South Asian'] = df['category'].str.contains('South Asian|Indian|Bangladeshi|Tandoori|Curry|Pakistani|Sri Lankan|Nepali')
df['South East Asian'] = df['category'].str.contains('South East Asian|Thai|Vietnamese|Malaysian|Indonesian|Filipino|Burmese')
df['American'] = df['category'].str.contains('Burger|American|Canadian|Hotdogs|Barbecue|Poke Bowl|Hawaiian')
df['Middle Eastern'] = df['category'].str.contains('Turkish|Kebab|Falafel|Halal|Middle Eastern|Lebanese|Israeli|Uzbek|Iraqi|Syrian|Egyptian|Iranian|Afghani')
df['Southern European'] = df['category'].str.contains('Spanish|Greek|Portuguese|Bulgarian|Balkan|Paella|Gyros|Mediterranean')
df['FrenchEuropean'] = df['category'].str.contains('French|Bistro|European|German|Dutch|Belgian|Scandinavian|Eastern European|Russian|Polish|Hungarian|Austrian|Swiss')
df['BritishIrish'] = df['category'].str.contains('British|Irish|Fish and Chips')
df['Latin American'] = df['category'].str.contains('Mexican|Burritos|Tacos|Nachos|Brazilian|Caribbean|Argentinian|Peruvian|South American|Chilean')
df['Bar'] = df['category'].str.contains('Wine|Bar|bars|Drinks|Beer|Tea|Coffee|Pub|Cafe')
df['African'] = df['category'].str.contains('African|Ethiopian|Nigerian|Kenyan|Algerian|Moroccan|Tunisian|Angolan|Senegalese')
df

,title,text,review date,rating,restaurant,location,total reviews,price,1-star,2-star,...,South Asian,South East Asian,American,Middle Eastern,Southern European,FrenchEuropean,BritishIrish,Latin American,Bar,African
0,AMAZING,I had the posh fish finger butty today it was ...,19 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,False,False,False,False,False,False,False,False,True,False
1,Blackdoor Bar & Cafe,Lovely little place just on my doorstep. Servi...,15 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,False,False,False,False,False,False,False,False,True,False
2,FREE COFFEES,I was working in the area due to a large fuel ...,15 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,False,False,False,False,False,False,False,False,True,False
3,Breakfast,"Great food, staff are very friendly and offer ...",14 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,False,False,False,False,False,False,False,False,True,False
4,Wow wow wow,After the last visit I thought I would call in...,14 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,Bit hipster but great,Star Wars arcade game and pinball and booze. W...,26 July 2015,5.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,False,False,False,False,False,False,True,False,True,False
1203,What a turnaround,I reviewed Brewdogs not so long ago and explai...,21 July 2015,5.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,False,False,False,False,False,False,True,False,True,False
1204,Meat eaters and vegans both satisfied,A group of us consisting of herbivores and omn...,13 July 2015,4.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,False,False,False,False,False,False,True,False,True,False
1205,A bit pricey but a nice change,I have been here a few times now and yes the p...,6 July 2015,4.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,False,False,False,False,False,False,True,False,True,False


In [76]:
# Export the scraped data to csv
df.to_csv('Tripadvisor_reviews_Liverpool.csv')

In [77]:
# Cuisine to one column

df['cuisine'] = np.where(df['Italian'] == True,'Italian',
        np.where(df['East Asian'] == True,'East Asian',
        np.where(df['South Asian'] == True,'South Asian',
        np.where(df['South East Asian'] == True,'South East Asian',
        np.where(df['American'] == True,'American',
        np.where(df['Latin American'] == True,'Latin American',
        np.where(df['FrenchEuropean'] == True,'French/European',
        np.where(df['Middle Eastern'] == True,'Middle Eastern',
        np.where(df['Southern European'] == True,'Southern European',
        np.where(df['BritishIrish'] == True,'Brisith/Irish',
        np.where(df['African'] == True,'African',
        np.where(df['Bar'] == True,'Bar/Drinks','Other'))))))))))))

# Check if restaurants are marked to have vegan, vega or gluten free options and if they're for breakfast or lunch
df['Vegan'] = df['category'].str.contains('Vegan')
df['Gluten Free'] = df['category'].str.contains('Gluten Free')
df['Vegetarian'] = df['category'].str.contains('Vegetarian')
df['Breakfast'] = df['category'].str.contains('Breakfast')
df['Lunch'] = df['category'].str.contains('Lunch')

In [78]:
# Get the smaller version (with no yes-no cuisine colunns)
df_new = pd.concat([df.iloc[:,0:16],df.iloc[:,28:]], axis = 1)
df_new.to_csv('Tripadvisor_cuisine_Liverpool.csv')
df_new

,title,text,review date,rating,restaurant,location,total reviews,price,1-star,2-star,...,5-star,float rating,category,avg_rating,cuisine,Vegan,Gluten Free,Vegetarian,Breakfast,Lunch
0,AMAZING,I had the posh fish finger butty today it was ...,19 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,104,4.94,['Bar' 'International' 'Pub'],5.0,Bar/Drinks,False,False,False,False,False
1,Blackdoor Bar & Cafe,Lovely little place just on my doorstep. Servi...,15 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,104,4.94,['Bar' 'International' 'Pub'],5.0,Bar/Drinks,False,False,False,False,False
2,FREE COFFEES,I was working in the area due to a large fuel ...,15 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,104,4.94,['Bar' 'International' 'Pub'],5.0,Bar/Drinks,False,False,False,False,False
3,Breakfast,"Great food, staff are very friendly and offer ...",14 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,104,4.94,['Bar' 'International' 'Pub'],5.0,Bar/Drinks,False,False,False,False,False
4,Wow wow wow,After the last visit I thought I would call in...,14 January 2023,5.0,Blackdoor Bar & Bistro,"19 Almonds Green, Liverpool L12 5HP England",108,££ - £££,0,1,...,104,4.94,['Bar' 'International' 'Pub'],5.0,Bar/Drinks,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,Bit hipster but great,Star Wars arcade game and pinball and booze. W...,26 July 2015,5.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,218,4.22,['Bar' 'British' 'Pub'],4.5,Brisith/Irish,False,False,False,False,False
1203,What a turnaround,I reviewed Brewdogs not so long ago and explai...,21 July 2015,5.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,218,4.22,['Bar' 'British' 'Pub'],4.5,Brisith/Irish,False,False,False,False,False
1204,Meat eaters and vegans both satisfied,A group of us consisting of herbivores and omn...,13 July 2015,4.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,218,4.22,['Bar' 'British' 'Pub'],4.5,Brisith/Irish,False,False,False,False,False
1205,A bit pricey but a nice change,I have been here a few times now and yes the p...,6 July 2015,4.0,BrewDog Liverpool,"8 Colquitt Street Manolis Yard, Liverpool L1 4...",354,££ - £££,5,9,...,218,4.22,['Bar' 'British' 'Pub'],4.5,Brisith/Irish,False,False,False,False,False


In [79]:
# Check the uncategorized cuisines
df[df['cuisine'] == 'Other']['category'].unique()

array([], dtype=object)

In [80]:
# Quit the Chrome driver
driver.quit()

In [73]:
def get_reviews(driver):
    i=1
    review_master = pd.DataFrame()
    while i <= 20: # either breaks before or at 20 pages of reviews 15*20 = 300 in total.
        try: # if the button "...More" is available click it.
            driver.find_element(By.XPATH,"//*[@class ='taLnk ulBlueLinks']").click()
        except: # otherwise the texts are short enough not needing such a button
            pass
        time.sleep(2)
        review = extract_reviews() # Get the reviews itself, see below
        review_master = pd.concat([review_master, review], axis = 0) # combine with already scraped reviews
        try: # navigate to the next page
            driver.find_elements(By.XPATH,"*//a[@class = 'nav next ui_button primary']")[0].click() 
            print('page',i)
            time.sleep(2)
        except:# Otherwise its the last page, it will tell then break.
            print('page',i,'is last')
            break
        i=i+1
    return(review_master)

In [11]:
def extract_reviews():
    reviews = pd.DataFrame()
    reviews['title'] = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"noQuotes")]
        
    splitted = [txt.text.split('\n') for txt in driver.find_elements(By.XPATH,"//div[contains(@class, 'ui_column is-9')]")[:15]]
    
    # We don't want answers from the restaurant, translations or others.
    revtxt = [np.array(split)[[spl != 'via mobile' for spl in split]][2] for split in splitted]
    reviews['text'] = np.array(revtxt)[[re.match('WHAT IS YOUR ANSWER|Answer|^AnswerShow|Google Translation',\
                                                 txt) == None for txt in revtxt]].tolist()
    
    reviews['review date'] = [txt.get_attribute("title") for txt in driver.find_elements(By.CLASS_NAME,"ratingDate")]

    # Get the bubble ratings elements value (rating)
    RevB = driver.find_elements(By.XPATH,"*//div[@class ='ui_column is-9']//span[contains(@class, 'ui_bubble_rating bubble_')]")

    reviews['rating'] = [int(re.findall('[0-9]+',txt.get_attribute("class"))[0])/10 for txt in RevB]
    
    try: # the restaurant is not at the expected place in very rare cases.
        reviews['restaurant'] = driver.find_element(By.CLASS_NAME,"HjBfq").text
    except:
        reviews['restaurant'] = driver.find_elements(By.CLASS_NAME,"breadcrumb")[-1].text
        
    reviews['location'] = [el.text for el in driver.find_elements(By.CLASS_NAME,"yEWoV")][0]
    reviews['total reviews'] = int(driver.find_elements(By.CLASS_NAME,"IcelI")[0].text.split(' ')[0]\
                                   .replace(',','').replace('.',''))
    
    # The cuisines, prices and extras may be in a different place. Can be these four.
    elementD = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"dlMOJ")]
    elementS = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"SrqKb")]
    elementA = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"AGRBq")]
    elementR = [txt.text for txt in driver.find_elements(By.CLASS_NAME,"RMOVd")]
    
    if len(np.array(elementD)[[re.match('[€,£,$]',txt) != None for txt in elementD]])>0:
        reviews['price'] = np.array(elementD)[[re.match('[€,£,$]',txt) != None for txt in elementD]][0]
    
    elif len(np.array(elementS)[[re.match('[€,£,$]',txt) != None for txt in elementS]])>0:
        reviews['price'] = np.array(elementS)[[re.match('[€,£,$]',txt) != None for txt in elementS]][0]
        
    elif len(np.array(elementA)[[re.match('[€,£,$]',txt) != None for txt in elementA]])>0:
        reviews['price'] = np.array(elementA)[[re.match('[€,£,$]',txt) != None for txt in elementA]][0]
    
    else: 'unknown'
                                        
    checkbox = [txt.text for txt in driver.find_elements(By.XPATH,"*//div[@class = 'ui_checkbox item']")]
    
    reviews['1-star'] = int(np.array(checkbox)[[re.match('Terrible',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['2-star'] = int(np.array(checkbox)[[re.match('Poor',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['3-star'] = int(np.array(checkbox)[[re.match('Average',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['4-star'] = int(np.array(checkbox)[[re.match('Very good',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['5-star'] = int(np.array(checkbox)[[re.match('Excellent',check) != None for check in checkbox]][0].split(' ')[-1]\
                                   .replace(',','').replace('.',''))
    reviews['float rating']   = round((reviews['1-star']*1+reviews['2-star']*2+reviews['3-star']*3+\
                                reviews['4-star']*4+reviews['5-star']*5)/reviews['total reviews'],2)
    
    if len(np.array(elementD)[[re.match('€',txt) == None for txt in elementD]])>0:
        reviews['category'] = np.array2string(np.array(elementD)\
                 [[re.match('[€,£,$]',txt) == None for txt in elementD]])
    
    elif len(np.array(elementS)[[re.match('€',txt) == None for txt in elementS]])>0:
        reviews['category'] = np.array(elementS)\
                 [[re.match('[€,£,$]',txt) == None for txt in elementS]][0]
    
    elif len(np.array(elementR)[[re.match('€',txt) == None for txt in elementR]])>0:
        reviews['category'] = np.array(elementR)\
                 [[re.match('[€,£,$]',txt) == None for txt in elementR]][0]
        
    elif len(np.array(elementA)[[re.match('€',txt) == None for txt in elementA]])>0:
        reviews['category'] = np.array2string(np.array(elementA[:-1])\
                 [[re.match('[€,£,$]',txt) == None for txt in elementA[:-1]]])
    else:
        reviews['category'] = 'unknown'
        
    RevA = driver.find_elements(By.XPATH,"*//span[@class ='MFMAn']//span[contains(@class, 'ui_bubble_rating bubble_')]")

    reviews['avg_rating'] = int(re.findall('[0-9]+',RevA[0].get_attribute("class"))[0])/10
    return(reviews)